In [3]:
import pandas as pd

In [4]:
from pathlib import Path

def read_data_files(folder: Path) -> pd.DataFrame:
    tmp_pandas_list = []

    for file in folder.glob('*.csv'):
        dataset = pd.read_csv(file, delimiter=',')

        tmp_pandas_list.append(dataset)

    # Concat csv files
    return pd.concat(tmp_pandas_list, axis=0, ignore_index=True)

In [7]:
data = read_data_files(Path('.'))

In [8]:
data

,test_name,loop counter,send time,send answer,message,trace hops,Source
0,marvin-wsl-11_52_02_002293.csv,0,0:00:00.000049,0:00:00.002072,27,4,marvin-wsl-11_52_02_002293
1,marvin-wsl-11_52_02_002293.csv,1,0:00:00.000038,0:00:00.001557,27,4,marvin-wsl-11_52_02_002293
2,marvin-wsl-11_52_02_002293.csv,2,0:00:00.000032,0:00:00.001668,27,4,marvin-wsl-11_52_02_002293
3,marvin-wsl-11_52_02_002293.csv,3,0:00:00.000033,0:00:00.001982,27,4,marvin-wsl-11_52_02_002293
4,marvin-wsl-11_52_02_002293.csv,4,0:00:00.000022,0:00:00.001708,27,4,marvin-wsl-11_52_02_002293
...,...,...,...,...,...,...,...
3310,marvin-wsl-11_52_28_838766.csv,995,0:00:00.000009,0:00:00.001918,912,9,marvin-wsl-11_52_28_838766
3311,marvin-wsl-11_52_28_838766.csv,996,0:00:00.000012,0:00:00.002436,912,9,marvin-wsl-11_52_28_838766
3312,marvin-wsl-11_52_28_838766.csv,997,0:00:00.000010,0:00:00.002354,912,9,marvin-wsl-11_52_28_838766
3313,marvin-wsl-11_52_28_838766.csv,998,0:00:00.000014,0:00:00.001962,912,9,marvin-wsl-11_52_28_838766
